In [ ]:
# cryptoデータの前処理をする
import polars as pl

import matplotlib.pyplot as plt

import stock

In [ ]:
# voluem barのcsvを作る
ticker = "BTC"
volume_size = 0.5

csv_lists = sorted((stock.PROJECT_ROOT / "data/tick_data").rglob("*_BTC.csv.gz"))
output_root_dir = stock.PROJECT_ROOT / "data/preprocessed/volume_bar/{}".format(str(volume_size).replace(".", "_"))
output_root_dir.mkdir(exist_ok=True)

last_row = None
for i in range(len(csv_lists)):
    csv_path = csv_lists[i]
    df = pl.read_csv(csv_path)
    volume_df = stock.io.volume_bar.convert_ticker_to_volume_bar(df, volume_size, last_row)

    if len(volume_df) == 0:
        continue

    if i < len(csv_lists) - 1:
        if volume_df["volume"][-1] < volume_size:
            last_row = volume_df[-1].to_numpy()[0]
            volume_df = volume_df[:-1]
        else:
            last_row = None
    output_path = output_root_dir / csv_path.name.replace(".csv.gz", "_{}.csv".format(str(volume_size).replace(".", "_")))
    volume_df.write_csv(output_path)